## Working With Sessions


Import the LArray library:


In [ ]:
from larray import *

### Before To Continue

If you not yet comfortable with creating, saving and loading sessions, please read first the [Creating Sessions](tutorial_presenting_larray_objects.ipynb#Creating-Sessions) and [Loading and Dumping Sessions](tutorial_IO.ipynb#Loading-and-Dumping-Sessions) sections of the tutorial before going further.  

### Exploring Content

To get the list of items names of a session, use the [names](../_generated/larray.Session.names.rst#larray.Session.names) shortcut (be careful that the list is sorted alphabetically and does not follow the internal order!):

In [ ]:
# load a session representing the results of a demographic model
filepath_hdf = get_example_filepath('demography_eurostat.h5')
demography_session = Session(filepath_hdf)

# print the content of the session
print(demography_session.names)

To get more information of items of a session, the [summary](../_generated/larray.Session.summary.rst#larray.Session.summary)  will provide not only the names of items but also the list of labels in the case of axes or groups and the list of axes, the shape and the dtype in the case of arrays:

In [ ]:
# print the content of the session
print(demography_session.summary())

### Selecting And Filtering Items

Session objects work like ordinary ``dict`` Python objects. To select an item, use the usual syntax ``<session_var>['<item_name>']``: 

In [ ]:
demography_session['population']

A simpler way consists in the use the syntax ``<session_var>.<item_name>``:

In [ ]:
demography_session.population

<div class="alert alert-warning">
    **Warning:** The syntax ``session_var.item_name`` will work as long as you don't use any special character like ``, ; :`` in the item's name.
</div>

To return a new session with selected items, use the syntax ``<session_var>[list, of, item, names]``:

In [ ]:
demography_session_new = demography_session['population', 'births', 'deaths']

demography_session_new.names

The [filter](../_generated/larray.Session.filter.rst#larray.Session.filter) method allows you to select all items of the same kind (i.e. all axes, or groups or arrays) or all items with names satisfying a given pattern:

In [ ]:
# select only arrays of a session
demography_session.filter(kind=Array)

In [ ]:
# selection all items with a name starting with a letter between a and k
demography_session.filter(pattern='[a-k]*')

### Iterating over Items

Like the built-in Python ``dict`` objects, Session objects provide methods to iterate over items:  

In [ ]:
# iterate over item names
for key in demography_session.keys():
    print(key)

In [ ]:
# iterate over items
for value in demography_session.values():
    if isinstance(value, Array):
        print(value.info)
    else:
        print(repr(value))
    print()

In [ ]:
# iterate over names and items
for key, value in demography_session.items():
    if isinstance(value, Array):
        print(key, ':')
        print(value.info)
    else:
        print(key, ':', repr(value))
    print()

### Arithmetic Operations On Sessions

Session objects accept binary operations with a scalar:

In [ ]:
# get population, births and deaths in millions
demography_session_div = demography_session / 1e6

demography_session_div.population

with an array (please read the documentation of the [random.choice](../_generated/larray.random.choice.rst#larray.random.choice) function first if you don't know it):

In [ ]:
from larray import random
random_increment = random.choice([-1, 0, 1], p=[0.3, 0.4, 0.3], axes=demography_session.population.axes) * 1000
random_increment

In [ ]:
# add some variables of a session by a common array
demography_session_rand = demography_session['population', 'births', 'deaths'] + random_increment

demography_session_rand.population

with another session:

In [ ]:
# compute the difference between each array of the two sessions
s_diff = demography_session - demography_session_rand

s_diff.births

### Applying Functions On All Arrays

In addition to the classical arithmetic operations, the [apply](../_generated/larray.Session.apply.rst#larray.Session.apply) method can be used to apply the same function on all arrays. This function should take a single element argument and return a single value:

In [ ]:
# add the next year to all arrays
def add_next_year(array):
    if 'time' in array.axes.names:
        last_year = array.time.i[-1] 
        return array.append('time', 0, last_year + 1)
    else:
        return array

demography_session_with_next_year = demography_session.apply(add_next_year)

print('population array before calling apply:')
print(demography_session.population)
print()
print('population array after calling apply:')
print(demography_session_with_next_year.population)

It is possible to pass a function with additional arguments:

In [ ]:
# add the next year to all arrays.
# Use the 'copy_values_from_last_year flag' to indicate 
# whether or not to copy values from the last year
def add_next_year(array, copy_values_from_last_year):
    if 'time' in array.axes.names:
        last_year = array.time.i[-1]
        value = array[last_year] if copy_values_from_last_year else 0
        return array.append('time', value, last_year + 1)
    else:
        return array

demography_session_with_next_year = demography_session.apply(add_next_year, True)

print('population array before calling apply:')
print(demography_session.population)
print()
print('population array after calling apply:')
print(demography_session_with_next_year.population)

It is also possible to apply a function on non-Array objects of a session. Please refer the documentation of the [apply](../_generated/larray.Session.apply.rst#larray.Session.apply) method.

### Comparing Sessions

Being able to compare two sessions may be useful when you want to compare two different models expected to give the same results or when you have updated your model and want to see what are the consequences of the recent changes.

[Session objects](../api.rst#session) provide the two methods to compare two sessions: [equals](../_generated/larray.Session.equals.rst#larray.Session.equals) and [element_equals](../_generated/larray.Session.element_equals.rst#larray.Session.element_equals):

-  The ``equals`` method will return True if **all items** from both sessions are identical, False otherwise.
-  The ``element_equals`` method will compare items of two sessions one by one and return an array of boolean values.

In [ ]:
# load a session representing the results of a demographic model
filepath_hdf = get_example_filepath('demography_eurostat.h5')
demography_session = Session(filepath_hdf)

# create a copy of the original session
demography_session_copy = demography_session.copy()

In [ ]:
# 'element_equals' compare arrays one by one
demography_session.element_equals(demography_session_copy)

In [ ]:
# 'equals' returns True if all items of the two sessions have exactly the same items
demography_session.equals(demography_session_copy)

In [ ]:
# slightly modify the 'population' array for some labels combination
demography_session_copy.population += random_increment 

In [ ]:
# the 'population' array is different between the two sessions
demography_session.element_equals(demography_session_copy)

In [ ]:
# 'equals' returns False if at least one item of the two sessions are different in values or axes
demography_session.equals(demography_session_copy)

In [ ]:
# reset the 'copy' session as a copy of the original session
demography_session_copy = demography_session.copy()

# add an array to the 'copy' session
demography_session_copy.gender_ratio = demography_session_copy.population.ratio('gender')

In [ ]:
# the 'gender_ratio' array is not present in the original session
demography_session.element_equals(demography_session_copy)

In [ ]:
# 'equals' returns False if at least one item is not present in the two sessions
demography_session.equals(demography_session_copy)

The ``==`` operator return a new session with boolean arrays with elements compared element-wise: 


In [ ]:
# reset the 'copy' session as a copy of the original session
demography_session_copy = demography_session.copy()

# slightly modify the 'population' array for some labels combination
demography_session_copy.population += random_increment

In [ ]:
s_check_same_values = demography_session == demography_session_copy

s_check_same_values.population

This also works for axes and groups:

In [ ]:
s_check_same_values.time

The ``!=`` operator does the opposite of ``==`` operator: 

In [ ]:
s_check_different_values = demography_session != demography_session_copy

s_check_different_values.population

A more visual way is to use the [compare](../_generated/larray.compare.rst#larray.compare) function which will open the ``Editor``.

```python
compare(demography_session, demography_session_alternative, names=['baseline', 'lower_birth_rate'])
```

![compare two sessions](../_static/compare_tutorial.png)

### Session API

Please go to the [Session](../api.rst#session) section of the API Reference to get the list of all methods of Session objects.